### Chromatin State
###### Pipeline to compare chromatin state between orthologous and non orthologous genes in two species. 

_Versions_
Bedtools: 2.29.0  
R: 3.6.1   
GNU Awk 4.0.2  
GNU Coreutils 8.22   
Python: 3.7.3  

In [ ]:
#Count how many total base pairs/genes from orthologous genes are within one of five chromatin states
bedtools intersect -wo -a dtri_lo_dmel_0.9r_final.GENES.bed -b Kc_chromatin_states_r6_muller.bed | sort -k8,8 | bedtools groupby -g 8 -c 9 -o sum | cut -f 2 | ./sum.pl
#Find the number and percent of orthologous genes in each chromatin state. 
bedtools intersect -wo -a dtri_lo_dmel_0.9r_final.GENES.bed -b Kc_chromatin_states_r6_muller.bed | sort -k8,8 | bedtools groupby -g 8 -c 9 -o sum | awk '{print $1"\t"$2"\t"$2/13180916}'
#Count how many total base pairs/genes from orthologous genes are within one of five chromatin states
bedtools intersect -wo -a dtri_lo_dmel_NOT_0.9r_final.GENES.bed -b Kc_chromatin_states_r6_muller.bed | sort -k8,8 | bedtools groupby -g 8 -c 9 -o sum | cut -f 2 | ./sum.pl
#Find the number and percent of orthologous genes in each chromatin state. 
bedtools intersect -wo -a dtri_lo_dmel_NOT_0.9r_final.GENES.bed -b Kc_chromatin_states_r6_muller.bed | sort -k8,8 | bedtools groupby -g 8 -c 9 -o sum | awk '{print $1"\t"$2"\t"$2/45589307}' 

Fisher's exact tests for state differences between orthologous and non-orthologous domains.

In [ ]:
import scipy.stats as stats
#BLACK
oddsratio, pvalue = stats.fisher_exact([[719,1653-719],[2156,7263-2156]],alternative='greater')

#BLUE
oddsratio, pvalue = stats.fisher_exact([[247,1653-247],[861,7263-861]],alternative='greater')

#GREEN
oddsratio, pvalue = stats.fisher_exact([[20,1653-20],[357,7263-357]],alternative='less')

#RED
oddsratio, pvalue = stats.fisher_exact([[116,1653-116],[526,7263-526]],alternative='less')

#YELLOW
oddsratio, pvalue = stats.fisher_exact([[551,1653-551],[3373,7263-3373]],alternative='less')